In [2]:
pip install pandas numpy matplotlib seaborn scikit-learn xgboost

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached xgboost-3.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.3.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------- -------------------------- 3.7/11.0 MB 19.9 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 29.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 24.0 MB/s  0:00:00
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ------------------- -------------------- 6.0/12.3 MB 34.9 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install streamlit plotly

  Using cached streamlit-1.52.2-py3-none-any.whl.metadata (9.8 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached altair-6.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.4-py3-none-any.whl.metadata (5.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached protobuf-6.33.2-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached gitpython-3.1.46-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


In [10]:
df = pd.read_csv("../data/Weekly_HospitaWeekly_Hospital_Respiratory_Data.csv")
df.head()


,Week Ending Date,Geographic aggregation,Number of Inpatient Beds,Number of Adult Inpatient Beds,Number of Pediatric Inpatient beds,Number of Inpatient Beds Occupied,Number of Adult Inpatient Beds Occupied,Number of Pediatric Inpatient Beds Occupied,Number of ICU Beds,Number of Adult ICU Beds,...,Absolute Change in the Percent Hospitals Reporting Percent Pediatric ICU Beds Occupied by RSV Patients from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Patients with COVID-19 in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Adult Patients with COVID-19 in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Pediatric Patients with COVID-19 in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Patients with Influenza in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Adult Patients with Influenza in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Pediatric Patients with Influenza in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Patients with RSV in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Adult Patients with RSV in the ICU from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Hospitalized Pediatric Patients with RSV in the ICU from Prior Week
0,2025-06-07,MA,15603.0,14281.0,1322.0,13138.0,12273.0,865.0,1601.0,1201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-07,OH,29105.0,25201.0,3883.0,20880.0,18664.0,2216.0,4538.0,3540.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-06-07,AZ,9115.0,8068.0,864.0,6575.0,5877.0,634.0,1548.0,1073.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-06-07,PR,7719.0,5073.0,675.0,3905.0,3610.0,286.0,532.0,497.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-06-07,SD,2582.0,2182.0,380.0,1750.0,1510.0,238.0,349.0,180.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for col in df.columns:
    print(col)


Week Ending Date
Geographic aggregation
Number of Inpatient Beds
Number of Adult Inpatient Beds
Number of Pediatric Inpatient beds
Number of Inpatient Beds Occupied
Number of Adult Inpatient Beds Occupied
Number of Pediatric Inpatient Beds Occupied
Number of ICU Beds
Number of Adult ICU Beds
Number of Pediatric ICU Beds
Number of ICU Beds Occupied
Number of Adult ICU Beds Occupied
Number of Pediatric ICU Beds Occupied
Number of Adult Patients Hospitalized with COVID-19
Number of Pediatric Patients Hospitalized with COVID-19
Total Patients Hospitalized with COVID-19
Number of  Adult Patients Hospitalized with Influenza
Number of Pedatric Patients Hospitalized with Influenza
Total Patients Hospitalized with Influenza
Number of Adult Patients Hospitalized with RSV
Number of Pediatric Patients Hospitalized with RSV
Total Patients Hospitalized with RSV
Number of Adult ICU Patients Hospitalized with COVID-19
Number of Pediatric ICU Patients Hospitalized with COVID-19
Total ICU Patients Hospi

In [12]:
id_cols = [
    "Week Ending Date",
    "Geographic aggregation"
]
capacity_cols = [
    "Number of ICU Beds",
    "Number of Adult ICU Beds",
    "Number of Inpatient Beds",
    "Number of Inpatient Beds Occupied",
    "Number of Adult Inpatient Beds Occupied"
]
stress_cols = [
    "Percent of Inpatient Beds Occupied",
    "Percent of Adult ICU Beds Occupied"
]

In [13]:
cols_to_keep = id_cols + capacity_cols + stress_cols

df_clean = df[cols_to_keep].copy()
df_clean.head()


KeyError: "['Percent of Inpatient Beds Occupied', 'Percent of Adult ICU Beds Occupied'] not in index"